For introduction and problem statement, please refer to notebook 1

## Content 

**Notebook 1: 1_cellphones_reviews_data_cleaning_and_eda**
- Data Import and Cleaning
- Exploratory Data Analysis
- Text Data Pre-processing

**Notebook 2: 2_cellphones_reviews_topic modelling**
- Data Import
- Topic Modelling with Gensim

**Notebook 3: 3_cellphones_reviews_topic_analysis_and_visualizations**
- Findings and Analysis of Topic Modelling

**Notebook 4: 4_features_extractions_and_sentiment_analysis**
- [Data Import](#Data-Import)
- [Sentiment Analysis with VADER](#Sentiment-Analysis-with-VADER)
- [entiment Analysis with Logistic Regression(Multi-Class Classification)](#Sentiment-Analysis-with-Logistic-Regression-Classifier)
- [Evaluation of Sentiment Analysis with BERT(Multi-Class Classification)](#Evaluation-of-Sentiment-Analysis-with-BERT)   
Please refer to notebook 5 for the fine-tuning process of pre-trained BERT model
- Comparison of the 3 Methods 
- Recommendation and Conclusion 
- Future Steps

**Notebook 5: fine_tuning_of_BERT_model**   
The reason why this notebook is separated from notebook 4 which contains the evaluation of BERT model is because the fine-tuning of BERT model requires GPU. Hence, the model was fine-tuned on Google Colaboratory and loaded back into notebook 4 for evaluation


## Data Import

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import spacy
from nltk import tokenize
from nltk.corpus import stopwords 
import re
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

In [3]:
new_reviews = pd.read_csv('../data/cleaned_combined_data.csv',na_filter=False)

## Sentiment Analysis with VADER

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [5]:
new_words = {
    'new': 3.0
}

analyser.lexicon.update(new_words)

In [6]:
stop_words = stopwords.words('english')

In [7]:
len(stop_words)

179

In [8]:
negation_words = ['ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', 
"hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 
'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
'wouldn', "wouldn't","not","no",'don',"don't"]

for word in negation_words:
    stop_words.remove(word)

len(stop_words)

139

In [9]:
def sentences_with_keywords (reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    summary = set()
    texts = tokenize.sent_tokenize(reviews)
    for sentence in texts:
        sentence = sentence.lower()
        for word in list_of_keywords:
            if word in sentence:
                summary.add(sentence)
                
    return list(summary)

In [10]:
def summarise_reviews (reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    summary = set()
    texts = tokenize.sent_tokenize(reviews)
    for sentence in texts:
        sentence = sentence.lower()
        for word in list_of_keywords:
            if word in sentence:
                # Remove HTML.
                post_text = BeautifulSoup(sentence).get_text()

                # Remove non-letters.
                letters_only = ' '.join(re.findall(r"[A-z’]+",post_text))

                # Convert to lower case, split into individual words.
                words = letters_only.lower().split()

                #convert the stopwords to a set.
                stops = set(stop_words)

                # Remove stopwords.
                meaningful_words = [w for w in words if w not in stops]

                # Stemming 
                #p_stemmer = PorterStemmer()
                #meaningful_words = [p_stemmer.stem(w) for w in meaningful_words]

                #Lemmatize
                lemmatizer = WordNetLemmatizer()
                meaningful_words = [lemmatizer.lemmatize(word) for word in meaningful_words]

                cleaned_sentence = (" ".join(meaningful_words))
                
                summary.add(cleaned_sentence)
                
    return list(summary)

In [110]:
def features_and_sentiments (summarised_reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    summary = set()
    
    for cleaned_sentence in summarised_reviews:
        
        for word in list_of_keywords:
            if word in cleaned_sentence:
                score = analyser.polarity_scores(cleaned_sentence)
                compound = score['compound']

                if compound >= 0.05:
                    sentiment_score = 5
                elif compound >= -0.05:
                    sentiment_score = 3
                else:
                    sentiment_score = 1

                summary.add((sentiment_score,word))
    return list(summary)

In [12]:
new_reviews['summary'] = new_reviews['reviews'].apply(summarise_reviews)

In [13]:
new_reviews['sentences_with_keywords'] = new_reviews['reviews'].apply(sentences_with_keywords)

In [111]:
new_reviews['vader_analysis'] = new_reviews['summary'].apply(features_and_sentiments)

In [15]:
pd.set_option('display.max_colwidth',None)
new_reviews[['reviews','summary']].sample(2)

,reviews,summary
5899,Great phone. Only thing is don't drop in in ... Great phone. Only thing is don't drop in in a cooler over night. LOL Buying our 2nd one today!,[]
20421,One Star Doesn't work well for hotspot/tethering,[]


In [16]:
new_reviews["filter summary"] = new_reviews['summary'].apply(lambda x: x != [])

In [17]:
new_reviews = new_reviews[new_reviews["filter summary"] == True]

In [18]:
new_reviews.shape

(22041, 25)

In [19]:
new_reviews.reset_index(inplace=True,drop=True)

In [20]:
new_reviews.head(2)

,asin,name,rating,date,verified,review_title,body,helpfulVotes,brand,item_title,...,originalPrice,reviews,word_count,cleaned_reviews,multi_class_sentiment,tokens,summary,sentences_with_keywords,features_and_sentiments,filter summary
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst","I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing. It finally died on me so I used this Nokia phone I bought in a garage sale for $1. I wonder y she sold it so cheap?... Bad: ===> I hate the menu. It takes forever to get to what you want because you have to scroll endlessly. Usually phones have numbered categories so u can simply press the # and get where you want to go. ===> It's a pain to put it on silent or vibrate. If you're in class and it rings, you have to turn it off immediately. There's no fast way to silence the damn thing. Always remember to put it on silent! I learned that the hard way. ===> It's so true about the case. It's a mission to get off and will break ur nails in the process. Also, you'll damage the case each time u try. For some reason the phone started giving me problems once I did succeed in opening it. ===> Buttons could be a bit bigger. Vibration could be stronger. Good: ===> Reception is not too shabby. I was using it in the elevator which is a remarkable feat considering my old phone would lose service by simply putting it in my pocket. ===> Compared to my old Samsung, this phone works quite well. The ring tones are loud enough to hear and the phone actually charges quickly and has great battery life. It doesn't heat up like a potatoe in the oven either during long phone convos. ===> Nice bright, large screen. ===> Cute ways to customize it. Scroll bar can be set to purple, pink, aqua, orange, etc. Overall: Okay phone. It serves its purpose but definitely pales in comparison to these new phones coming out from Sprint. Why get so so when you can get great?",1.0,,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice Activated Dialing & Bright White Backlit Screen,...,0.0,"Def not best, but not worst I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing. It finally died on me so I used this Nokia phone I bought in a garage sale for $1. I wonder y she sold it so cheap?... Bad: ===> I hate the menu. It takes forever to get to what you want because you have to scroll endlessly. Usually phones have numbered categories so u can simply press the # and get where you want to go. ===> It's a pain to put it on silent or vibrate. If you're in class and it rings, you have to turn it off immediately. There's no fast way to silence the damn thing. Always remember to put it on silent! I learned that the hard way. ===> It's so true about the case. It's a mission to get off and will break ur nails in the process. Also, you'll damage the case each time u try. For some reason the phone started giving me problems once I did succeed in opening it. ===> Buttons could be a bit bigger. Vibration could be stronger. Good: ===> Reception is not too shabby. I was using it in the elevator which is a remarkable feat considering my old phone would lose service by simply putting it in my pocket. ===> Compared to my old Samsung, this phone works quite well. The ring tones are loud enough to hear and the phone actually charges quickly and has great battery life. It doesn't heat up like a potatoe in the oven either during long phone convos. ===> Nice bright, large screen. ===> Cute ways to customize it. Scroll bar can be set to purple, pink, aqua, orange, etc. Overall: Okay phone. It serves its purpose but definitely pales in comparison to these new phones coming out from Sprint. Why get so so when you can get great?",333,def best worst samsung awhile absolute doo doo read review detect rage stupid thing finally died used nokia bought garage sale wonder sold cheap bad hate menu take forev

In [21]:
#new_reviews.to_csv('../data/cleaned_combined_data_with_keywords.csv',index=False)

In [22]:
#iphone_xs = new_reviews[new_reviews['asin'] == 'B07RT1X4FJ']

In [23]:
#pd.set_option('display.max_colwidth',None)
#iphone_xs['reviews']

In [24]:
#iphone_xs['summary'][63325]

In [25]:
#pd.set_option('display.max_colwidth',None)
#iphone_xs['features_and_sentiments']

In [26]:
#iphone_xs.loc[63202,'features_and_sentiments']

## Sentiment Analysis with Logistic Regression Classifier

In [27]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
new_reviews['multi_class_sentiment'].unique()

array([1, 2, 0])

In [29]:
# Create the feature and target variable
X = new_reviews['cleaned_reviews']
y = new_reviews['multi_class_sentiment']

In [30]:
# Create train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    random_state = 42,
                                                    stratify= y)

In [31]:
y_train.value_counts(normalize=True)

2    0.633575
0    0.269933
1    0.096491
Name: multi_class_sentiment, dtype: float64

In [32]:
baseline_model = y_train.value_counts(normalize=True)
baseline_accuracy = round(baseline_model[2],3)

print(f"Baseline accuracy: {baseline_accuracy}")

Baseline accuracy: 0.634


In [33]:
#Instantiate the pipeline
lr_cvec_pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('lr',LogisticRegression(random_state=42,solver='liblinear', max_iter=10000))
])

#create hyperparameters for gridsearch
lr_cvec_params = {
    'cvec__max_features': [3000,4000,5000],
    'cvec__min_df':[2,3],
    'cvec__max_df':[0.9,0.95],
    'cvec__ngram_range': [(1,1), (1,2)],
    'lr__C':[0.01,0.1,1],
    'lr__penalty': ['l1', 'l2']
}

# Instantiate GridSearchCV.
lr_cvec_gs = GridSearchCV(lr_cvec_pipe, # what object are we optimizing?
                  param_grid=lr_cvec_params , # what parameters values are we searching?
                  cv=5,
                 n_jobs=-1,verbose=1) 

#fit the model
lr_cvec_gs.fit(X_train,y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  7.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('cvec',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

In [34]:
training_accuracy_score = round(lr_cvec_gs.score(X_train,y_train),3)
testing_accuracy_score = round(lr_cvec_gs.score(X_test,y_test),3)

print(f"Logistic Regression CVEC Train Accuracy Score: {training_accuracy_score}")
print(f"Logistic Regression CVEC Test Accuracy Score: {testing_accuracy_score}")

Logistic Regression CVEC Train Accuracy Score: 0.888
Logistic Regression CVEC Test Accuracy Score: 0.834


In [35]:
import pickle
filename= '../data/logreg_3classes.pkl'
pickle.dump(lr_cvec_gs,open(filename,'wb'))

### Predictions on Feature Level with Logistic Regression

In [102]:
def logreg_classification (reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    summary = set()
    pred = lr_cvec_gs.predict(reviews)
    
    predicted_ratings= []
    for score in pred:
        if float(score) == 2.0:
            rating = 5
            predicted_ratings.append(rating)
        elif float(score) == 1.0:
            rating = 3
            predicted_ratings.append(rating)
        else:
            rating = 1
            predicted_ratings.append(rating)
    
    
    for i,cleaned_sentence in enumerate(reviews):        
        for word in list_of_keywords:
            if word in cleaned_sentence:
                summary.add((predicted_ratings[i],word))
                
    
    return list(summary)
logreg_classification (new_reviews['summary'][0])

[(5, 'battery'), (5, 'screen')]

In [103]:
new_reviews['logreg_pred'] = new_reviews['summary'].apply(logreg_classification)

## Evaluation of Sentiment Analysis with BERT

In [40]:
from transformers import BertForSequenceClassification
import torch
from tqdm.notebook import tqdm

In [41]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [42]:
len(new_reviews['multi_class_sentiment'].unique())

3

In [43]:
X_train, X_val, y_train, y_val = train_test_split(new_reviews.index.values,
                                                  new_reviews.multi_class_sentiment.values,
                                                  test_size = 0.25,
                                                  random_state= 42,
                                                  stratify=new_reviews.multi_class_sentiment.values)

In [44]:
y_train.shape

(16530,)

In [45]:
y_val.shape

(5511,)

In [46]:
new_reviews['data_type'] = ['not_set']*new_reviews.shape[0]

In [47]:
new_reviews.loc[X_train, 'data_type'] = 'train'
new_reviews.loc[X_val, 'data_type'] = 'val'

In [48]:
new_reviews.head(2)

,asin,name,rating,date,verified,review_title,body,helpfulVotes,brand,item_title,...,word_count,cleaned_reviews,multi_class_sentiment,tokens,summary,sentences_with_keywords,features_and_sentiments,filter summary,logreg_pred,data_type
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst","I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing. It finally died on me so I used this Nokia phone I bought in a garage sale for $1. I wonder y she sold it so cheap?... Bad: ===> I hate the menu. It takes forever to get to what you want because you have to scroll endlessly. Usually phones have numbered categories so u can simply press the # and get where you want to go. ===> It's a pain to put it on silent or vibrate. If you're in class and it rings, you have to turn it off immediately. There's no fast way to silence the damn thing. Always remember to put it on silent! I learned that the hard way. ===> It's so true about the case. It's a mission to get off and will break ur nails in the process. Also, you'll damage the case each time u try. For some reason the phone started giving me problems once I did succeed in opening it. ===> Buttons could be a bit bigger. Vibration could be stronger. Good: ===> Reception is not too shabby. I was using it in the elevator which is a remarkable feat considering my old phone would lose service by simply putting it in my pocket. ===> Compared to my old Samsung, this phone works quite well. The ring tones are loud enough to hear and the phone actually charges quickly and has great battery life. It doesn't heat up like a potatoe in the oven either during long phone convos. ===> Nice bright, large screen. ===> Cute ways to customize it. Scroll bar can be set to purple, pink, aqua, orange, etc. Overall: Okay phone. It serves its purpose but definitely pales in comparison to these new phones coming out from Sprint. Why get so so when you can get great?",1.0,,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice Activated Dialing & Bright White Backlit Screen,...,333,def best worst samsung awhile absolute doo doo read review detect rage stupid thing finally died used nokia bought garage sale wonder sold cheap bad hate menu take forever get want scroll endlessly usually phone numbered category u simply press get want go pain put silent vibrate class ring turn immediately fast way silence damn thing always remember put silent learned hard way true case mission get break ur nail process also damage case time u try reason started giving problem succeed opening button could bit bigger vibration could stronger good reception shabby using elevator remarkable feat considering old would lose service simply putting pocket compared old samsung work quite well ring tone loud enough hear actually charge quickly great battery life heat like potatoe oven either long convos nice bright large screen cute way customize scroll bar set purple pink aqua orange etc overall okay serf purpose definitely pale comparison new phone coming sprint get get great,1,"['def', 'best', 'worst', 'samsung', 'awhile', 'absolute', 'doo', 'doo', 'read', 'review', 'detect', 'rage', 'stupid', 'thing', 'finally', 'died', 'used', 'nokia', 'bought', 'garage', 'sale', 'wonder', 'sold', 'cheap', 'bad', 'hate', 'menu', 'take', 'forever', 'get', 'want', 'scroll', 'endlessly', 'usually', 'phone', 'numbered', 'category', 'u', 'simply', 'press', 'get', 'want', 'go', 'pain', 'put', 'silent', 'vibrate', 'class', 'ring', 'turn', 'immediately', 'fast', 'way', 'silence', 'damn', 'thing', 'always', 'remember', 'put', 'silent', 'learned', 'hard', 'way', 'true', 'case', 'mission', 'get', 'break', 'ur', 'nail', 'process', 'also', 'damage', 'case', 'time', 'u', 'try', 'reason', 'started', 'giving', 'problem', 'succeed', 'opening', 'button', 'could', 'bit', 'bigger', 'vibration', 'could', 'stronger', 'good', 'reception', 'shabby', 'using', 'elevator', 'remarkable', 'feat', 'considerin

In [49]:
new_reviews.groupby(['rating', 'data_type']).count()

asin  name  date  verified  review_title  body  \
rating data_type                                                   
1      train      3049  3049  3049      3049          3049  3049   
       val        1035  1035  1035      1035          1035  1035   
2      train      1413  1413  1413      1413          1413  1413   
       val         452   452   452       452           452   452   
3      train      1595  1595  1595      1595          1595  1595   
       val         532   532   532       532           532   532   
4      train      2754  2754  2754      2754          2754  2754   
       val         877   877   877       877           877   877   
5      train      7719  7719  7719      7719          7719  7719   
       val        2615  2615  2615      2615          2615  2615   

                  helpfulVotes  brand  item_title   url  ...  reviews  \
rating data_type                                         ...            
1      train              3049   3049        3049  3049  ...     3049   
       val                1035   1035        1035  1035  ...     1035   
2      train              1413   1413        1413  1413  ...     1413   
       val                 452    452         452   452  ...      452   
3      train              1595   1595        1595  1595  ...     1595   
       val                 532    532         532   532  ...      532   
4      train              2754   2754        2754  2754  ...     2754   
       val                 877    877         877   877  ...      877   
5      train              7719   7719        7719  7719  ...     7719   
       val                2615   2615        2615  2615  ...     2615   

                  word_count  cleaned_reviews  multi_class_sentiment  tokens  \
rating data_type                                                               
1      train            3049             3049                   3049    3049   
       val              1035             1035                   1035    1035   
2      train            1413             1413                   1413    1413   
       val               452              452                    452     452   
3      train            1595             1595                   1595    1595   
       val               532              532                    532     532   
4      train            2754             2754                   2754    2754   
       val               877              877                    877     877   
5      train            7719             7719                   7719    7719   
       val              2615             2615                   2615    2615   

                  summary  sentences_with_keywords  features_and_sentiments  \
rating data_type                                                              
1      train         3049                     3049                     3049   
       val           1035                     1035                     1035   
2      train         1413                     1413                     1413   
       val            452                      452                      452   
3      train         1595                     1595                     1595   
       val            532                      532                      532   
4      train         2754                     2754                     2754   
       val            877                      877                      877   
5      train         7719                     7719                     7719   
       val           2615                     2615                     2615   

                  filter summary  logreg_pred  
rating data_type                               
1      train                3049         3049  
       val                  1035         1035  
2      train                1413         1413  
       val                   452          452  
3      train                1595         1595  
       val                   532          532  
4      train                27

## Loading Tokenizer

In [50]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [51]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [52]:
#prepare the data in a format that is readable by BERT
#since we are not doing fine-tuning of the train data here
#we will only prepare the validation data for evaluation
#the full-fine tuning process including tokening and dataloader 
#of train dataset is available in notebook 5

encoded_data_val = tokenizer.batch_encode_plus(
    new_reviews[new_reviews.data_type=='val'].reviews.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val= torch.tensor(new_reviews[new_reviews.data_type=='val'].multi_class_sentiment.values)

In [53]:
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [54]:
len(dataset_val)

5511

## Setting up BERT Pretrained Model

In [55]:
from transformers import BertForSequenceClassification

In [56]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)


## Creating Data Loaders

In [57]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [58]:
batch_size = 32


dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

## Defining our Performance Metrics

In [59]:
def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    correct_pred = 0
    total_count = 0
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        
        correct_pred = correct_pred + len(y_preds[y_preds==label])
        total_count = total_count + len(y_true)
        
    print(f'Total Accuracy:{correct_pred/total_count}' )

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [60]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [66]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [67]:
model.load_state_dict(torch.load('../data/finetuned_BERT_epoch_2_3classes.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [68]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [69]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 1374/1487

Class: 1
Accuracy: 93/532

Class: 2
Accuracy: 3296/3492

Total Accuracy:0.8642714570858283


## Predictions on Feature Level with BERT

### Loading Tokenizer and Encoding Data by Sentences

In [70]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [71]:
def bert_sentiments (summarised_reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    
    summary = set()
    
    encoded_data_features = tokenizer.batch_encode_plus(
    summarised_reviews, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

    input_ids_features = encoded_data_features['input_ids']
    attention_masks_features = encoded_data_features['attention_mask']
    #labels_features = torch.tensor(df[df.data_type=='val'].label.values)

    dataset_features = TensorDataset(input_ids_features, attention_masks_features)

    dataloader_features = DataLoader(dataset_features , 
                                       sampler=SequentialSampler(dataset_features ), 
                                       batch_size=batch_size)

    
    model.eval()


    for batch in dataloader_features:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                     }

    with torch.no_grad():        
        outputs = model(**inputs)
        
        
    
    rating_score = torch.argmax(outputs[0],dim=1)

    
    try:
        
        predicted_ratings = []

        for score in rating_score:
            if float(score) == 2.0:
                rating = 5
                predicted_ratings.append(rating)
            elif float(score) == 1.0:
                rating = 3
                predicted_ratings.append(rating)
            else:
                rating = 1
                predicted_ratings.append(rating)

        for i,cleaned_sentence in enumerate(summarised_reviews):        
            for word in list_of_keywords:
                if word in cleaned_sentence:
                    summary.add((float(predicted_ratings[i]),word))
    except:
        summary.add(np.nan)
        
                
    
    return summary



bert_sentiments(new_reviews.sentences_with_keywords.values[5]) 

{(1.0, 'battery'),
 (1.0, 'camera'),
 (5.0, 'battery'),
 (5.0, 'camera'),
 (5.0, 'ringtones'),
 (5.0, 'screen')}

In [74]:
new_reviews['features_and_sentiments'][5]

[(3, 'camera'),
 (3, 'battery'),
 (5, 'screen'),
 (5, 'camera'),
 (2, 'ringtones'),
 (5, 'battery')]

In [72]:
tqdm.pandas()

/Users/elisenerissa/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [76]:
new_reviews['bert_analysis'] = new_reviews['sentences_with_keywords'].progress_apply(bert_sentiments)

In [107]:
new_reviews['bert_analysis'] = new_reviews['bert_analysis'].map(lambda x:list(x))

In [113]:
new_reviews.to_csv("../data/reviews_with_feature_sentiments.csv",index=False)

In [114]:
new_reviews.to_pickle("../data/reviews_with_feature_sentiments.pkl")

## Mean ratings by features of each unique product

In [ ]:
new_reviews.reset_index(inplace=True,drop=True)

In [115]:
unique_asins = new_reviews['asin'].unique()

In [116]:
new_reviews.loc[1,'features_and_sentiments']

[(3, 'ringtones')]

In [117]:
all_products = {}

#for cell in new_review['features_and_sentiments']: 
all_features=set()

for product in unique_asins:
    all_products[product] = {'camera':[],'battery':[],'fingerprint':[],'screen':[],'charger':[]}
    for idx in new_reviews.index:
        if new_reviews.loc[idx,'asin'] == product:
            for feature in new_reviews.loc[idx,'features_and_sentiments']:
                all_features.add(feature[1])
                if feature[1] =='battery':
                    all_products[product]['battery'].append(feature[0])
                elif feature[1]  == 'camera':
                    all_products[product]['camera'].append(feature[0])
                elif feature[1]  == 'charger':
                    all_products[product]['charger'].append(feature[0])
                elif feature[1] == 'screen':
                    all_products[product]['screen'].append(feature[0])
                elif feature[1] == 'fingerprint':
                    all_products[product]['fingerprint'].append(feature[0])
        

In [118]:
all_features

{'battery',
 'camera',
 'charger',
 'fingerprint',
 'ringtones',
 'screen',
 'simcard',
 'touchscreen'}

In [ ]:
for key_1,value_1 in all_products.items():
    for key_2,value_2 in all_products[key_1].items():
        try:
            all_products[key_1][key_2] = round(np.mean(all_products[key_1][key_2]),1)
        except:
            all_products[key_1][key_2] = np.nan

In [ ]:
mean_ratings = pd.DataFrame(all_products).T

In [ ]:
mean_ratings.reset_index(inplace=True)
mean_ratings

In [ ]:
mean_ratings.rename(columns={'index':'asin'},inplace=True)

In [ ]:
updated_mean_ratings = pd.merge(mean_ratings,new_reviews[['asin','item_title']],on='asin',how='inner')
updated_mean_ratings.drop_duplicates(subset=['asin'],keep='first',inplace=True)

In [ ]:
updated_mean_ratings.reset_index(inplace=True,drop=True)

In [ ]:
updated_mean_ratings.tail(20)

In [ ]:
## Mean ratings by features of each unique product

new_reviews.reset_index(inplace=True,drop=True)

unique_asins = new_reviews['asin'].unique()

new_reviews.loc[1,'features_and_sentiments']


all_products = {}

#for cell in new_review['features_and_sentiments']: 
all_features=set()

for product in unique_asins:
    all_products[product] = {'camera':[],'battery':[],'fingerprint':[],'screen':[],'charger':[]}
    for idx in new_reviews.index:
        if new_reviews.loc[idx,'asin'] == product:
            for feature in new_reviews.loc[idx,'features_and_sentiments']:
                all_features.add(feature[1])
                if feature[1] =='battery':
                    all_products[product]['battery'].append(feature[0])
                elif feature[1]  == 'camera':
                    all_products[product]['camera'].append(feature[0])
                elif feature[1]  == 'charger':
                    all_products[product]['charger'].append(feature[0])
                elif feature[1] == 'screen':
                    all_products[product]['screen'].append(feature[0])
                elif feature[1] == 'fingerprint':
                    all_products[product]['fingerprint'].append(feature[0])
        

for key_1,value_1 in all_products.items():
    for key_2,value_2 in all_products[key_1].items():
        try:
            all_products[key_1][key_2] = round(np.mean(all_products[key_1][key_2]),1)
        except:
            all_products[key_1][key_2] = np.nan

mean_ratings = pd.DataFrame(all_products).T

mean_ratings.reset_index(inplace=True)
mean_ratings

mean_ratings.rename(columns={'index':'asin'},inplace=True)

updated_mean_ratings = pd.merge(mean_ratings,new_reviews[['asin','item_title']],on='asin',how='inner')
updated_mean_ratings.drop_duplicates(subset=['asin'],keep='first',inplace=True)

updated_mean_ratings.reset_index(inplace=True,drop=True)

updated_mean_ratings.tail(20)